In [1]:
# prepend parent dir to import from local pcsp
import sys
sys.path.insert(0, '../')

In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pcsp
from pcsp import PCSPipeline, ModuleSet, Module, init_args, convert
from pcsp.pipeline import build_graph
from functools import partial
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.decomposition import PCA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ind = {}
def top_n_features(X, Y, n, i):
    if i not in ind:
        corr = np.abs(np.apply_along_axis(lambda x: np.corrcoef(x, Y[:,i])[0,1], 0, X))
        ind[i] = np.argsort(corr[~np.isnan(corr)])[::-1][:n]
    return X[:,ind[i]]

def pca(X, n):
    return PCA(n_components=n, copy=True).fit(X).transform(X)


In [4]:
# load data
X = np.asarray(pd.read_csv("../data/fmri/fit_feat.txt", sep=" ", error_bad_lines=False))
Y = np.asarray(pd.read_csv("../data/fmri/resp_dat.txt", sep=" ", error_bad_lines=False))

In [5]:
np.random.seed(14)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test), names=['X_train', 'X_test', 'y_train', 'y_test'])

# split y_train by voxel and extract top 500 correlated features per voxel
voxel_extract_funcs = [partial(lambda x, y, i: (top_n_features(x, y, 20, i), y[:,i]), i=i) for i in range(20)]
voxel_extract_set = ModuleSet(name='voxel_extract', modules=voxel_extract_funcs)
X_trains, y_trains = voxel_extract_set(X_train, y_train)
X_tests, y_tests = voxel_extract_set(X_test, y_test)

/Users/rush/anaconda3/envs/binyugroup/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/rush/anaconda3/envs/binyugroup/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
# modeling
modeling_set = ModuleSet(name='modeling', modules=[LassoCV()], module_keys=["Lasso"])
modeling_set.fit(X_trains, y_trains)

In [11]:
# predict matches any X_tests key ('X_test', 'y_test', 'voxel_extract_i') with 
# all modeling_set.out keys ('X_train', 'y_train', 'voxel_extract_j', 'Lasso')
preds = modeling_set.predict(X_trains, match_on=2)
# output keys are form (('X_train', 'y_train', 'voxel_extract_0', 'Lasso'), ('X_test', 'y_test', 'voxel_extract_0'))

In [14]:
# evaluate fails to match y_tests key ('X_test', 'y_test', 'voxel_extract_i') with preds key 
# (('X_train', 'y_train', 'voxel_extract_i', 'Lasso'), ('X_test', 'y_test', 'voxel_extract_i'))
hard_metrics_set = ModuleSet(name='hard_metrics', modules=[r2_score, explained_variance_score], module_keys=["R2", "EV"])
hard_metrics = hard_metrics_set.evaluate(y_trains, preds)

In [15]:
hard_metrics

{(('X_train', 'y_train', 'voxel_extract_0', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_0'),
  'R2'): 0.17080031040166133,
 (('X_train', 'y_train', 'voxel_extract_0', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_0'),
  'EV'): 0.17080031040166122,
 (('X_train', 'y_train', 'voxel_extract_1', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_1'),
  'R2'): 0.2158212567120792,
 (('X_train', 'y_train', 'voxel_extract_1', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_1'),
  'EV'): 0.2158212567120792,
 (('X_train', 'y_train', 'voxel_extract_2', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_2'),
  'R2'): 0.14764006625321402,
 (('X_train', 'y_train', 'voxel_extract_2', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_2'),
  'EV'): 0.14764006625321413,
 (('X_train', 'y_train', 'voxel_extract_3', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_3'),
  'R2'): 0.16691584031264028,
 (('X_train', 'y_train', 'voxel_extract_3', 'Lasso'),
  ('X_train', 'y_train', 'voxel_extract_3'),
